In [1]:
import ANNarchy as ann
import numpy as np
ann.clear()

ANNarchy 4.6 (4.6.9.3) on darwin (posix). 


In [2]:
Izhikevich_Hamker = ann.Neuron(
parameters="""
    a = 0.02
    b = 0.2
    c = -72.0
    d = 6.0
    n0 = 140.
    n1 = 5.0
    n2 = 0.04
    I = 0.0
    tau_refrac = 10.0
    tau_ampa = 10.0
    tau_gaba = 10.0
    E_ampa = 0.0
    E_gaba = -90.0
    tau_syn = 1.0
    C = 1.0
    v_th = 30.0
""",
equations="""
    I_syn_ex = - g_ampa*(v-E_ampa)
    I_syn_in = - g_gaba*(v-E_gaba)
    I_syn = I_syn_ex + I_syn_in - g_base*v
    dg_base/dt = -g_base/tau_syn : init = 0
    dg_ampa/dt = -g_ampa/tau_ampa : init = 0
    dg_gaba/dt = -g_gaba/tau_gaba : init = 0
    dv/dt = n2*v*v+n1*v+n0 - u/C  + I + I_syn : init = -70.
    du/dt = a*(b*(v)-u) : init = -18.55
""",
spike = """
    v>=v_th
""",
reset = """
    v = c
    u = u+d
""",
refractory = """tau_refrac"""
)

In [3]:
try:
    import shutil
    shutil.rmtree("./annarchy")
    print("Deleted previously compiled network!")
except:
    print("No previously compiled network to delete!")

Deleted previously compiled network!


In [4]:
nNeurons = 1000
rate = 12 # Hz
simLength = 10000 # ms

import pickle

filename = "poisson_"+str(nNeurons)+"_n_"+str(int(rate))+"_Hz_"+str(simLength)+"_ms.txt"

with open (filename, 'rb') as fp:
    spike_times = pickle.load(fp)


In [5]:

stimE1 = ann.SpikeSourceArray(spike_times=spike_times)

exc1 = ann.Population(nNeurons, neuron=Izhikevich_Hamker)
proj = ann.Projection(pre=stimE1, post=exc1, target="ampa")
proj.connect_one_to_one(weights=1.0)

In [6]:
# this is for checking the number of neurons in the generated spike source array
#ann.Global._network[0]["populations"][1].geometry

In [7]:
record_from = ["v", "u", "I_syn", "I_syn_ex", "I_syn_in", "g_base", "g_ampa", "g_gaba"]
n_vars = len(record_from)
exc1_m = ann.Monitor(exc1, record_from, period=1.0) 
exc1_spikes_rec = ann.Monitor(exc1, "spike")
inp_spikes_rec = ann.Monitor(stimE1, "spike")

In [ ]:
#ann.compile("-march=native -O2 --clean")
ann.compile()

In [ ]:
simulation_time = 10000.0 # ms
ann.simulate(simulation_time)

In [ ]:
ts = exc1_m.times()
exc1_ts = exc1_m.get()
spikes = exc1_spikes_rec.get("spike")
spike_times, spike_senders = exc1_spikes_rec.raster_plot(spikes)

spike_times2, spike_senders2 = inp_spikes_rec.raster_plot(spikes)

In [ ]:
from matplotlib import pyplot
pyplot.close("all")
rate = len(spike_times) / simulation_time * 1000
f1 = pyplot.figure(figsize=(10, 2.5))
pyplot.plot(spike_times, spike_senders, ".")
pyplot.xlabel("Time (ms)")
pyplot.title("%g (Hz)" % rate)
pyplot.show()


f3 = pyplot.figure(figsize=(10, 2.5))
pyplot.plot(spike_times2, spike_senders2, ".")
pyplot.xlabel("Time (ms)")
pyplot.title("%g (Hz)" % rate)
pyplot.show()


f2, axes = pyplot.subplots(n_vars, 1, sharex=True, figsize=(10, 10))
for i_var, var in enumerate(record_from):
    varts = exc1_ts[var]
    axes[i_var].plot(np.arange(ts[var]["start"][0], len(varts)*exc1_m.period, exc1_m.period), varts)
    axes[i_var].set_title(var)
axes[-1].set_xlabel("Time (ms)")
pyplot.show()

In [ ]:
params = {}
for attr in exc1.attributes:
    params[attr] = getattr(exc1, attr)
print (params)